# Data Cleaning

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Load data 
raw_data = pd.read_csv("Dataset/kirana.csv")
raw_data["timestamp"] = pd.to_datetime(raw_data["timestamp"])
raw_data = raw_data.set_index('timestamp').sort_index()

raw_data.head(5)

,tx_id,store_type,total_amount,payment_method,tendered_amount,tendered_breakdown,change_given,change_breakdown,notes
timestamp,,,,,,,,,
2025-08-18 01:42:12,e869d89d-9af3-4a67-be4a-a9b3463cf1a0,kirana,119.36,cash,150.00,"{""100"":1,""50"":1}",30.64,"{""20"":1,""10"":1}",NaN
2025-08-18 03:32:40,35ed6ae3-0753-4491-ae67-1ad812353e39,kirana,97.16,cash,97.16,"{""50"":1,""20"":2,""5"":1,""2"":1}",0.00,NaN,NaN
2025-08-18 06:00:29,f3c2c91d-40f0-454c-8b16-f22fcac23089,kirana,5.36,cash,5.36,"{""5"":1}",0.00,NaN,NaN
2025-08-18 07:09:04,84c422e9-20c0-4edb-9b9c-e913d317a8f7,kirana,205.37,cash,205.37,"{""200"":1,""5"":1}",0.00,NaN,NaN
2025-08-18 07:11:48,4b277543-7e9f-413c-ab1f-469921a016e0,kirana,6.49,cash,20.00,"{""20"":1}",13.51,"{""10"":1,""2"":1,""1"":1}",NaN


In [3]:
# Preparing Cash data

data = raw_data[raw_data['payment_method']=='cash'].resample('h').size().to_frame('count')

data = data.asfreq('h',fill_value = 0)

data.head(6)

,count
timestamp,
2025-08-18 01:00:00,1
2025-08-18 02:00:00,0
2025-08-18 03:00:00,1
2025-08-18 04:00:00,0
2025-08-18 05:00:00,0
2025-08-18 06:00:00,1


# Feature Engineering

In [4]:
data['lag_24'] = data['count'].shift(24)
data['hour'] = data.index.hour
data['dayofweek'] = data.index.dayofweek
data = data.dropna()

data

,count,lag_24,hour,dayofweek
timestamp,,,,
2025-08-19 01:00:00,0,1.0,1,1
2025-08-19 02:00:00,0,0.0,2,1
2025-08-19 03:00:00,0,1.0,3,1
2025-08-19 04:00:00,1,0.0,4,1
2025-08-19 05:00:00,0,0.0,5,1
...,...,...,...,...
2025-11-15 16:00:00,0,0.0,16,5
2025-11-15 17:00:00,4,6.0,17,5
2025-11-15 18:00:00,2,7.0,18,5


In [5]:
#Split data for testing and training
split = int(len(data) * 0.8)
train = data.iloc[:split]
test = data.iloc[split:].copy()

# Model Training

In [6]:
model = xgb.XGBRegressor(n_estimators=100, random_state=42)
model.fit(train[['hour', 'dayofweek', 'lag_24']], train['count'])

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [7]:
test['pred_count'] = model.predict(test[['hour', 'dayofweek', 'lag_24']])

In [8]:
# 6. Apply "Daily Average" Logic
# Calculate the daily average for ACTUAL data (The Truth)
daily_avg_actual = test.groupby(test.index.date)['count'].transform('mean')

# Calculate the daily average for PREDICTED data (The Model's guess)
daily_avg_pred = test.groupby(test.index.date)['pred_count'].transform('mean')

# Define "Busy" as being above that day's average
test['busy_actual'] = (test['count'] > daily_avg_actual).astype(int)
test['busy_predicted'] = (test['pred_count'] > daily_avg_pred).astype(int)

# Evaluation

In [9]:
cm = confusion_matrix(test['busy_actual'], test['busy_predicted'])
precision = precision_score(test['busy_actual'], test['busy_predicted'])
recall = recall_score(test['busy_actual'], test['busy_predicted'])

print("--- Evaluation Results ---")
print(f"Precision (Trustworthiness): {precision:.2%}")
print(f"Recall (Hit Rate): {recall:.2%}")
print("\nConfusion Matrix:")
print(cm)
print("\n[True Negatives  False Alarms]")
print("[Missed Spikes   True Spikes ]")

--- Evaluation Results ---
Precision (Trustworthiness): 91.41%
Recall (Hit Rate): 90.00%

Confusion Matrix:
[[286  11]
 [ 13 117]]

[True Negatives  False Alarms]
[Missed Spikes   True Spikes ]


In [10]:
# 1. Select only the columns we care about
results_table = test[['count', 'pred_count', 'busy_actual', 'busy_predicted']].copy()

# 2. Rename them to be human-readable
results_table.columns = [
    'Actual Cash Txns', 
    'Predicted Cash Txns', 
    'Is Actual Busy Hour?', 
    'Is Predicted Busy Hour?'
]

# 3. Formatting: Round the predicted numbers to 1 decimal place
results_table['Predicted Cash Txns'] = results_table['Predicted Cash Txns'].round(1)

# 4. Map the "1" and "0" to "Yes" and "No" for easier reading
results_table['Is Actual Busy Hour?'] = results_table['Is Actual Busy Hour?'].map({1: 'YES', 0: '-'})
results_table['Is Predicted Busy Hour?'] = results_table['Is Predicted Busy Hour?'].map({1: 'YES', 0: '-'})

# --- Display the Table ---

# Option A: Print the last 24 hours of data to the screen
print("\n--- Prediction Results (Last 24 Hours) ---")
results_table.tail(24)


--- Prediction Results (Last 24 Hours) ---


,Actual Cash Txns,Predicted Cash Txns,Is Actual Busy Hour?,Is Predicted Busy Hour?
timestamp,,,,
2025-11-14 21:00:00,0,0.2,-,-
2025-11-14 22:00:00,0,0.5,-,-
2025-11-14 23:00:00,0,0.0,-,-
2025-11-15 00:00:00,0,-0.1,-,-
2025-11-15 01:00:00,0,-0.0,-,-
2025-11-15 02:00:00,1,0.3,-,-
2025-11-15 03:00:00,0,0.1,-,-
2025-11-15 04:00:00,0,0.3,-,-
2025-11-15 05:00:00,0,0.5,-,-


In [11]:
import joblib
# 'model' is the variable name of your XGBRegressor
joblib.dump(model, 'model_spikes.pkl')

['model_spikes.pkl']